<a href="https://colab.research.google.com/github/adsk2050/btp-iitg/blob/master/2DFEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# https://colab.research.google.com/drive/1Uu3jHPlzxGlKdwh_bfVM6q5we9J-jvtH

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files
import csv
import numpy as np
import scipy as sp
from scipy.stats import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import pandas as pd
import shutil, os
np.set_printoptions(precision=1)


class Material:
    def __init__(self, c11, c12, c21, c22, c66):
        """ creates D matrix """
        self.prop = np.array([[c11, c12,   0],
                              [c21, c22,   0],
                              [  0,   0, c66]])
        
class Node:
    def __init__(self, x, y, gi):
        """ gi is the global index of the node """
        self.x = x
        self.y = y
        self.gi =gi

class Element:
    """ We don't have body force, so """
    on_boundary=False
    nodes = np.zeros((4,3))
    u = np.zeros((1, 4))
    v = np.zeros((1, 4))
    def __init__(self, node_list):
        """ Assuming node_list contains 4 nodes in anticlockwise order """
        for i in range(0, 4):
            self.nodes[i][0]=node_list[i].x
            self.nodes[i][1]=node_list[i].y
            self.nodes[i][2]=node_list[i].gi
    
    def isOnBoundary(self):
        self.on_boundary=True
    
    def sizni(self, z, n):
        """ Approximation function for x and y in terms of z and n """
        si1zn = 0.25*(1-z)*(1-n)
        si2zn = 0.25*(1+z)*(1-n)
        si3zn = 0.25*(1+z)*(1+n)
        si4zn = 0.25*(1-z)*(1+n)
        self.si = np.array([si1zn, si2zn, si3zn, si4zn])

    def dsiidz(self, z, n):
        """ Derivative of approximation fn wrt z """
        dsi1dz = -0.25*(1-n)
        dsi2dz = 0.25*(1-n)
        dsi3dz = 0.25*(1+n)
        dsi4dz = -0.25*(1+n)
        self.dsidz = np.array([dsi1dz, dsi2dz, dsi3dz, dsi4dz])
    
    def dsiidn(self, z, n):
        """ Derivative of approximation fn wrt n """
        dsi1dn = -0.25*(1-z)
        dsi2dn = -0.25*(1+z)
        dsi3dn = 0.25*(1+z)
        dsi4dn = 0.25*(1-z)
        self.dsidn = np.array([dsi1dn, dsi2dn, dsi3dn, dsi4dn])

    def Je(self, z, n):
        """ Jacobian """
        x = []
        y = []
        for i in self.nodes:
            x.append(i[0])
            y.append(i[1])
        dxdz = 0.25*(n*(x[0]-x[1]+x[2]+x[3])+(-x[0]+x[1]+x[2]-x[3]))
        dxdn = 0.25*(z*(x[0]-x[1]+x[2]-x[3])+(-x[0]-x[1]+x[2]+x[3]))
        dydz = 0.25*(n*(y[0]-y[1]+y[2]+y[3])+(-y[0]+y[1]+y[2]-y[3]))
        dydn = 0.25*(z*(y[0]-y[1]+y[2]-y[3])+(-y[0]-y[1]+y[2]+y[3]))
        self.J =  np.array([[dxdz, dxdn],
                            [dydz, dydn]])
    
    def diff(self):
        """ for each iteration  """
        try:
            Jstar = np.linalg.inv(self.J)
        except np.linalg.LinAlgError:
            Jstar = np.zeros((2, 2))
            print("The Jacobian is singular, change element size or something")
        tmp = [[self.dsidz[0]],
               [self.dsidn[0]]]
        [[dsi1dx],[dsi1dy]] = np.dot(Jstar, tmp)

        tmp = [[self.dsidz[1]],
               [self.dsidn[1]]]
        [[dsi2dx],[dsi2dy]] = np.dot(Jstar, tmp)

        tmp = [[self.dsidz[2]],
               [self.dsidn[2]]]
        [[dsi3dx],[dsi3dy]] = np.dot(Jstar, tmp)

        tmp = [[self.dsidz[3]],
               [self.dsidn[3]]]
        [[dsi4dx],[dsi4dy]] = np.dot(Jstar, tmp)
        self.dsidx = np.array([dsi1dx, dsi2dx, dsi3dx, dsi4dx])
        self.dsidy = np.array([dsi1dy, dsi2dy, dsi3dy, dsi4dy])
    
    def makeB(self):
        """ creates B matrix for [K] = integral [B]t[D][B]det([J])dzdn which is stiffness matrix (8*8) """
        self.B = np.array([[self.dsidx[0],             0, self.dsidx[1],             0, self.dsidx[2],             0, self.dsidx[3],             0],
                           [            0, self.dsidy[0],             0, self.dsidy[1],             0, self.dsidy[2],             0, self.dsidy[3]], 
                           [self.dsidy[0], self.dsidx[0], self.dsidy[1], self.dsidx[1], self.dsidy[2], self.dsidx[2], self.dsidy[3], self.dsidx[3]]])

    def makeK(self, z, n, mat):
        """
        Will do numerical integration with only one quadrature point
        n=z=0 and W=4
        """
        pts, wts = np.polynomial.legendre.leggauss(2)
        _K = np.zeros((8, 8))
        for i in range(0, len(wts)):
            for j in range(0, len(wts)):
                z = pts[i]
                n = pts[j]
                wi = wts[i]
                wj = wts[j]
                self.sizni(z, n)
                self.dsiidz(z, n)
                self.dsiidn(z, n)
                self.Je(z, n)
                self.diff()
                self.makeB()
                detJ = np.linalg.det(self.J)
                _K += wi*wj*detJ*(np.dot(np.transpose(self.B), np.dot(mat.prop, self.B))) 
        # K = np.zeros((8, 8))
        # j=0
        # for i in range(0,4):
        #     K[j]=_K[2*i]
        #     K[j+1]=_K[2*i+1]
        #     j+=2
        self.K=_K

    # def makeFQ(self, z, n, mat, Forces):
    #     """
    #     Will do numerical integration with only one quadrature point
    #     n=z=0 and W=4

    #     Also, need to find tx and ty
    #     """
    #     self.sizni(z, n)
    #     self.dsiidz(z, n)
    #     self.dsiidn(z, n)
    #     self.Je(z, n)
    #     self.diff()
    #     self.FQ = np.zeros((8,0))
    #     if self.on_boundary==False:
    #         pass
    #     else:
    #         j=0
    #         for i in range(0,4):
    #             FQ[i]=tx*self.si[i]
    #             FQ[2*i]=ty*self.si[i]   

def load_data():
    node_list=[]
    with open('/content/drive/My Drive/Colab Notebooks/nodes.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=',')
        for row in csv_reader:
            tmp = Node(float(row[1]), float(row[2]), int(row[0]))
            node_list.append(tmp)
        f.close()

    _CM = []
    with open('/content/drive/My Drive/Colab Notebooks/connectivity.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=',')
        for row in csv_reader:
            _CM.append([int(ri) for ri in row])
        f.close()

    no_elements = len(_CM)
    no_nodes = len(node_list)
    CM = np.zeros((no_elements, 4))#8))
    for element in _CM:
        CM[element[0]-1] = element[1:5]
    
    L = np.zeros((no_elements, 8))
    a=[]
    j=1
    for i in range(1, no_nodes+1):
        a.append([j, j+1])
        j+=2

    for i in range(0, no_elements):
        k=0
        for j in range(0, 4):
            L[i][k:k+2]=a[int(CM[i][j])-1]
            k+=2
    return node_list, CM, L

def Assemble(CM, L, node_list, mat, length, breadth, height):
    """
    For rectangular linear element (=> with 4 nodes per element), No. of nodes = 4*ne
    Connectivity matrix CM

    """
    K_list = []
    # FQ_list = []
    # element_list = []
    for elt in CM: #as CM=matrix, elt=row of matrix which is itself a list
        is_onboundary=False
        local_node_list = []
        for gi in elt: # 
            ni = node_list[int(gi)-1]
            if ni.x==length or ni.y==breadth:
                is_onboundary
            local_node_list.append(ni)
        element = Element(local_node_list)
        element.makeK(0, 0, mat)
        # element.makeFQ(0, 0, mat)
        a = element.K*height
        K_list.append(a)
        print(a)
        if is_onboundary:
            element.isOnBoundary()
        # element_list.append(element)
        # FQ_list.append(element.FQ)
        
    no_nodes = len(node_list)
    K = np.zeros((2*no_nodes, 2*no_nodes))
    no_elements = len(K_list)
    for i in range(0, no_elements):
        Ki = K_list[i]
        for j in range(0, 8):
            for k in range(0, 8):
                l=int(L[i][j])-1
                m=int(L[i][k])-1
                K[l][m]+=Ki[j][k]
    return K#, element_list#, FQ

def ApplyBC(K, node_list, F0, height):
    """ Apply Dirichlet (displacement) and Neuman(force) boundary conditions """
    no_nodes = len(node_list)
    F = np.zeros((2*no_nodes, 1))
    for i in range(0, no_nodes):
        node = node_list[i]
        print(node.x, " ",  node.y, "\n")
        if node.x==0:
            K[i, :]=0
            K[:, i]=0
            K[i, i]=1
            K[i+1, :]=0
            K[:, i+1]=0
            K[i+1, i+1]=1
        # elif node.x==0:
        #     K[i, :]=0
        #     K[:, i]=0
        #     K[i, i]=1
        # elif node.y==0:
        #     K[i+1, :]=0
        #     K[:, i+1]=0
        #     K[i+1, i+1]=1
        if node.x==10:
            if node.y>10:
                F[i+1]=F0
            elif node.y<10:
                F[i+1]=-1*F0
    return F, K

def savePlots(node_list, U):
    x = []
    y = []
    u=[]
    v=[]
    for i in range(0, len(node_list)):
        node = node_list[i]
        x.append(node.x)
        y.append(node.y)
        u.append(U[2*i])
        v.append(U[2*i+1])
    # print(u, '\n', v)
    plt.quiver(x,y, u, v)
    plt.savefig('test{}.jpg'.format(i))
    files.download('test{}.jpg'.format(i))
    plt.clf()

node_list, CM, L= load_data()
print(L)
print(CM)
length = 10
breadth = 20
height = 5e-9
F0 = 100
E=210e9
nu=0.3
# for plane stress
c11 = E/(1-nu*nu)
c12 = (nu*E)/(1-nu*nu)
c21=c12
c22 = c11
c66 = E/(2*(1+nu))
mat = Material(c11, c12, c21, c22, c66)
K = Assemble(CM, L, node_list, mat, length, breadth, height)
df1 = pd.DataFrame(K)
no_elements = len(K)
F, K = ApplyBC(K, node_list, F0, height)
U = np.linalg.solve(K, F).flatten()
savePlots(node_list, U)
print(U)

df2 = pd.DataFrame(K)
df1.subtract(df2, fill_value=0) 
with open('dataa.csv', 'w') as f:
    f.write(df1.to_csv(index=False))
f.close()

files.download('dataa.csv') 

[[ 1.  2.  3.  4.  9. 10.  7.  8.]
 [ 3.  4.  5.  6. 11. 12.  9. 10.]
 [ 7.  8.  9. 10. 15. 16. 13. 14.]
 [ 9. 10. 11. 12. 31. 32. 15. 16.]
 [13. 14. 15. 16. 21. 22. 19. 20.]
 [15. 16. 17. 18. 23. 24. 21. 22.]
 [19. 20. 21. 22. 27. 28. 25. 26.]
 [21. 22. 23. 24. 29. 30. 27. 28.]]
[[ 1.  2.  5.  4.]
 [ 2.  3.  6.  5.]
 [ 4.  5.  8.  7.]
 [ 5.  6. 16.  8.]
 [ 7.  8. 11. 10.]
 [ 8.  9. 12. 11.]
 [10. 11. 14. 13.]
 [11. 12. 15. 14.]]
[[ 631.4  312.5 -362.2 -139.4 -282.1 -187.5   12.8   14.4]
 [ 312.5  839.7 -110.6  -70.5 -187.5 -323.7  -14.4 -445.5]
 [-362.2 -110.6  496.8  -62.5  147.4  -14.4 -282.1  187.5]
 [-139.4  -70.5  -62.5  455.1   14.4  -60.9  187.5 -323.7]
 [-282.1 -187.5  147.4   14.4  496.8   62.5 -362.2  110.6]
 [-187.5 -323.7  -14.4  -60.9   62.5  455.1  139.4  -70.5]
 [  12.8  -14.4 -282.1  187.5 -362.2  139.4  631.4 -312.5]
 [  14.4 -445.5  187.5 -323.7  110.6  -70.5 -312.5  839.7]]
[[1168.3  500.  -899.  -326.9 -389.4 -250.   120.2   76.9]
 [ 500.  1168.3 -298.1 -399.  -250

<Figure size 432x288 with 0 Axes>

In [0]:
print(K)
print(F)

[[1.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 1.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 1.7e+03 ... 0.0e+00 0.0e+00 0.0e+00]
 ...
 [0.0e+00 0.0e+00 0.0e+00 ... 1.3e+03 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 4.2e+02 6.2e+01]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 6.2e+01 4.9e+02]]
[[  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [100.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]]


In [0]:
import numpy as np

# n1 = Node(0, 0, 1)
# n2 = Node(2, 0, 2)
# n3 = Node(2, 2, 3)
# n4 = Node(0, 2, 4)
# node_list = [n1, n2, n3, n4]
# el1 = Element(node_list)

print("Nodes\n", el1.nodes)
z=0
n=0
# el1.sizni(z, n)
# el1.dsiidz(z, n)
# el1.dsiidn(z, n)
# el1.Je(z, n)
# el1.diff()
el1.makeK(z, n, mat)
print("si values:\n", el1.si)
print("d(si)/dz\n", el1.dsidz)
print("d(si)/dn\n", el1.dsidn)
print("d(si)/dx\n", el1.dsidx)
print("d(si)/dy\n", el1.dsidy)
print("Jacobian\n", el1.J)
print("B matrix\n", el1.B)
print("K matrix\n", el1.K)
print(np.shape(el1.K))

E = 210e9
print(E)
a = np.array([[1, -1],
              [-1, 1]])
try:
    inverse = np.linalg.inv(a)
except np.linalg.LinAlgError:
    inverse = np.zeros((2, 2))
print(a)
b = np.array([[5],
              [6]])
count=0
for i in range(0, 2):
    for j in range(0, 2):
        a[i][j]=count
        count+=1
print(a)

q = np.zeros((4, 4))
print(q)

210000000000.0
[[ 1 -1]
 [-1  1]]
[[0 1]
 [2 3]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [0]:
def setk(self, mat):
    """ Stiffness matrix with material properties stored in mat(6x6) """
    k11=np.zeros((4, 4))
    for i in range(0, 4):
        for j in range(0, 4):
            k[i][j] = mat.prop[0][0]*self.dsidx[i]*self.dsidx[j]+mat.prop[2][2]*self.dsidy[i]*dsidy[j]
    
    k12=np.zeros((4, 4))
    for i in range(0, 4):
        for j in range(0, 4):
            k[i][j] = mat.prop[0][1]*self.dsidx[i]*self.dsidy[j]+mat.prop[2][2]*self.dsidy[i]*dsidy[j]
    
    k21=np.zeros((4, 4))
    for i in range(0, 4):
        for j in range(0, 4):
            k[i][j] = mat.prop[1][0]*self.dsidy[i]*self.dsidx[j]+mat.prop[2][2]*self.dsidx[i]*dsidy[j]
    
    k22=np.zeros((4, 4))
    for i in range(0, 4):
        for j in range(0, 4):
            k[i][j] = mat.prop[1][1]*self.dsidy[i]*self.dsidy[j]+mat.prop[2][2]*self.dsidx[i]*dsidx[j]


    print(k11)
    print(k12)
    print(k21)
    print(k22)



In [0]:
import numpy as np
# import scipy as sp

a = np.polynomial.hermite.hermgauss(2)
print(a)
a = "Anmol{0}".format(i)
print(a)
"""
*Node
1,           0.,           0.
2,  0.200000003,           0.
3,  0.400000006,           0.
4,  0.600000024,           0.
5,  0.800000012,           0.
6,           1.,           0.
7,           0.,         0.25
8,  0.200000003,         0.25
9,  0.400000006,         0.25
10,  0.600000024,         0.25
11,  0.800000012,         0.25
12,           1.,         0.25
*Element, type=CPS4R
1,  1,  2,  8,  7
2,  2,  3,  9,  8
3,  3,  4, 10,  9
4,  4,  5, 11, 10
5,  5,  6, 12, 11
"""
# a=[]
# j=1
# for i in range(1, 11):
#     a.append([j, j+1])
#     j+=2

# print(a)

a = np.array([[1, 2, 0],
              [0, 0, 0]])

b = a.flatten()
print(b)
# print([i.all()==0 for i in a])

# print("Connectivity matrix\n", CM)


# with open('foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat /content/drive/My\ Drive/foo.txt
print(int(True))

no_elements = 12
print(no_elements)
for k in range(0, no_elements):
    print(k)

(array([-0.7,  0.7]), array([0.9, 0.9]))
Anmol11
[1 2 0 0 0 0]
1
12
0
1
2
3
4
5
6
7
8
9
10
11
